In [ ]:
# import modules for Qiskit
import qiskit
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister, BasicAer, Aer, IBMQ, execute, schedule
from qiskit.providers.aer.noise import NoiseModel
from qiskit.tools.visualization import plot_histogram
from qiskit.tools.monitor import job_monitor
from qiskit.providers.aer import AerSimulator
from qiskit.compiler import transpile
from qiskit.transpiler import PassManager

import numpy as np
from numpy import pi
np.set_printoptions(precision=3)

import pandas as pd
import sys, hashlib, os, random, time, math
from datetime import date

if sys.version_info < (3, 6):
	import sha3

# code for getting the backend
print("Getting provider...")
IBMQ.load_account()
provider = IBMQ.get_provider(hub='ibm-q-internal', group='deployed', project='default')

# choose the hardware
backend_name = 'ibmq_auckland'
backend = provider.get_backend(backend_name)
simulator_name = 'qasm_simulator'

# parameters
numberOfcalcs = 5
MAX_NONCE = 2**32
previous_hash = '' # empty hash
block_number = 0
transactions ='Schroedinger paid Einstein 1 qBTC'
n_qreg_list = [18] # list of the numbers of qubits to experiment with
num_intersection = 100 # don't change this - how many different numbers of intersection to keep track of
qubit_arr = [20, 19, 16, 14, 11, 8, 5, 3, 2, 1, 4, 7, 10, 12, 15, 18, 21, 23] # specify which qubits to use
error_list = [None]*len(n_qreg_list) # list of errors for each number of qubits

# create data files
today = date.today().strftime("%Y%m%d")
filepath = os.path.join(os.getcwd(), 'Results') #specify data storage folder
filename_log = os.path.join(filepath, today + f'_results_{n_qreg_list[0]}qubits.txt')

# QPoW cycle: function takes the input text (block number, nonce,...) and pushes it through once 

def qPoW(text, quantum_circuit, errors, num_states, num_intersection):
    """
    qPoW takes the input text and pushes it through once, and it updates the array of errors

    :param text: the text that is hashed and used to parameterize the quantum circuit
    :param quantum_circuit: the function to build the parameterized quantum circuit
    :param errors: the array of errors for different percentages of maxstates and overlap requirements
    :param num_states: the maximum number of states considered
    :param num_intersection: the number of overlap percentages considered (from 0% to 100%)
    :return: the updated error array
    """
    hashIn = hashlib.sha3_256(text.encode("ascii")).hexdigest() # hashing the 'text' input
    str1 = 'hashIn-hex: ' + str(hashIn) + ' length ' + str(len(hashIn))
    print(str1, '\n')

    # convert hashIn(hex) to hashIn_bin(binary)
    scale = 16 # hex base
    hashIn_bin = bin(int(hashIn, scale))[2:].zfill(len(hashIn)*4)
    str2 = 'hashIn-binary: ' + str(hashIn_bin) + ' length ' + str(len(hashIn_bin))
    print(str2, '\n')

    # comparison of the results produced by quantum simulator and quantum computer
    qstate_bin_sim = sim_quantum_operation(quantum_circuit, hashIn_bin, num_states)
    qstate_bin_exp = exp_quantum_operation(quantum_circuit_verification, hashIn_bin, num_states) #qstate is a 256 binary number

    # error rate calculation
    for states in range(1, num_states+1):
        maxstates_sim = set(qstate_bin_sim[:states])
        maxstates_exp = set(qstate_bin_exp[:states])
        overlap_val = math.ceil(overlap(maxstates_sim, maxstates_exp)*num_intersection/states+1e-20)
        errors[states][overlap_val:num_intersection+1]+=1
    
    maxstates_sim = set(qstate_bin_sim[:num_states])
    maxstates_exp = set(qstate_bin_exp[:num_states])
    num_overlap = overlap(maxstates_sim, maxstates_exp)

    return errors, str1, str2, num_overlap

def overlap(maxstates_sim, maxstates_exp):
    """
    overlap calculates the overlap between two sets

    :param maxstates_sim: the set of states from the simulation
    :param maxstates_exp: the set of states from the noise simulator or quantum computer
    :return: the number of overlapping states
    """
    return len(maxstates_sim.intersection(maxstates_exp))
    
# converting hashIn_bin to a bit string to pass thru a quantum processor
def break_up_4bit_values(hashIn_bin):
    """
    break_up_4bit_values converts the input into an array of 4-bit strings

    :param hashIn_bin: the binary string that is broken up
    :return: array of 4-bit strings
    """
    array_4_bit_values = []

    for i in range(64): 
      four_bits = hashIn_bin[2+4*i:2+4*i+4]
      array_4_bit_values.append(four_bits)
        
    print("hashIn binary split into 4bit bins:", array_4_bit_values)
    return array_4_bit_values

def quantum_circuit(q_par, n_qreg, circ_layer = 1):
    """
    quantum_circuit builds a paramterized quantum circuit

    :param q_par: the parameters
    :param n_qreg: the number of qubits
    :param circ_layer: the number of times to repeat the circuit
    :return: the quantum circuit
    """
    k = 0 # counter for the parameter values
    
    # circuit 15
    # setting the quantum circuit:
    n = min(24, backend.configuration().n_qubits)
    qreg_q = QuantumRegister(n, 'q')
    creg_c = ClassicalRegister(n, 'c')
    circuit = QuantumCircuit(qreg_q, creg_c)
    for layer in range(circ_layer):

        for i in range(n_qreg):   
            circuit.ry(pi/2, qreg_q[qubit_arr[i]])

        for i in range(0, n_qreg):
            circuit.crx(q_par[k+i]*pi/8, qreg_q[qubit_arr[n_qreg-i-1]], qreg_q[qubit_arr[(n_qreg-i)%n_qreg]])
        k+=n_qreg
        
        for i in range(n_qreg):
            circuit.ry(pi/2, qreg_q[qubit_arr[i]])
        
        for i in range(0, n_qreg):
            circuit.crx(q_par[k+i]*pi/8, qreg_q[qubit_arr[(n_qreg-1+i)%n_qreg]], qreg_q[qubit_arr[(n_qreg-2+i)%n_qreg]])
        k+=n_qreg

    # measurements of all qubits
    for i in range(n_qreg):
        circuit.measure(qreg_q[qubit_arr[i]], creg_c[i])

    return circuit

def quantum_circuit_verification(q_par, n_qreg, circ_layer = 1):
    """
    quantum_circuit_verification builds a paramterized quantum circuit for verification

    :param q_par: the parameters
    :param n_qreg: the number of qubits
    :param circ_layer: the number of times to repeat the circuit
    :return: the quantum circuit
    """
    k = 0 # counter for the parameter values
    
    # circuit 15
    # setting the quantum circuit:
    qreg_q = QuantumRegister(n_qreg, 'q')
    creg_c = ClassicalRegister(n_qreg, 'c')
    circuit = QuantumCircuit(qreg_q, creg_c)
    for layer in range(circ_layer):

        for i in range(n_qreg):   
            circuit.ry(pi/2, qreg_q[i])

        for i in range(0, n_qreg):
            circuit.crx(q_par[k+i]*pi/8, qreg_q[n_qreg-i-1], qreg_q[(n_qreg-i)%n_qreg])
        k+=n_qreg
        
        for i in range(n_qreg):
            circuit.ry(pi/2, qreg_q[i])
        
        for i in range(0, n_qreg):
            circuit.crx(q_par[k+i]*pi/8, qreg_q[(n_qreg-1+i)%n_qreg], qreg_q[(n_qreg-2+i)%n_qreg])
        k+=n_qreg

    # measurements of all qubits
    for i in range(n_qreg):
        circuit.measure(qreg_q[i], creg_c[i])

    return circuit


def exp_quantum_operation(quantum_circuit, hashIn, num_states):
    """
    exp_quantum_operation runs the paramterized circuit with either a noise simulator or a real quantum computer

    :param quantum_circuit: function that returns the desired quantum circuit
    :param hashIn: the SHA3-256 hash of the input string
    :param num_states: the maximum number of states considered
    :return: the top num_states output states in an array
    """
    # input hashIn string
    fourbit_array = break_up_4bit_values(hashIn)
    q_par = [int(fourbit_array[i],2) for i in range(len(fourbit_array)-1)] #throwing away the last string element
    circuit = quantum_circuit(q_par, n_qreg)
    
    transpiled_circuit = transpile(circuit, backend, seed_transpiler=13)
    qpu_job = backend.run(transpiled_circuit, shots=20000)
    job_id = qpu_job.job_id()
    job_monitor(qpu_job)
    results = qpu_job.result()
    print("Time taken: " + str(results.time_taken) + '\n')

    counts = results.get_counts(circuit)

    # picking up the maximally probable states
    max_state256 = sorted(counts, key=counts.get, reverse=True)[:num_states]
    
    return max_state256 # 4bit vector

# quantum simulator run
def sim_quantum_operation(quantum_circuit, hashIn, num_states):
    """
    sim_quantum_operation runs the paramterized circuit with the qasm simulator

    :param quantum_circuit: function that returns the desired quantum circuit
    :param hashIn: the SHA3-256 hash of the input string
    :param num_states: the maximum number of states considered
    :return: the top num_states output states in an array
    """
    # input hashIn string
    fourbit_array = break_up_4bit_values(hashIn)
    q_par = [int(fourbit_array[i],2) for i in range(len(fourbit_array)-1)] #throwing away the last string element
    circuit = quantum_circuit(q_par, n_qreg)

    backend = BasicAer.get_backend(simulator_name) # run on local simulator by default 

    job = execute(circuit, backend, shots=20000)

    # Monitor job progress and wait until complete:
    job_monitor(job)

    # Get the job results (this method also waits for the job to complete):
    results = job.result()
    print("Verification time taken: " + str(results.time_taken) + '\n')
    
    counts = results.get_counts(circuit)

    # picking up the maximally probable states
    max_state256 = sorted(counts, key=counts.get, reverse=True)[:num_states]

    return max_state256 # 4bit vector

# main method
if(not os.path.isdir(os.path.join(os.getcwd(), 'Results'))):
    os.mkdir(os.path.join(os.getcwd(), 'Results'))
for i in range(len(n_qreg_list)):
    n_qreg = n_qreg_list[i]
    num_states = int(.01*2**n_qreg)
    errors = np.zeros((num_states+1, num_intersection+1)) # number of 'False' nonces
    
    for counter in range(1, numberOfcalcs+1):
        
        print(f'run {counter}')
        
        # Execute both exp and qPoW
        nonce = random.randint(0, MAX_NONCE)
        print('nonce:', nonce, '\n') 
        text = str(block_number) + transactions + previous_hash + str(nonce) #hash input
        print('text:', text, '\n')

        errors, hash_hex, hash_binary, num_overlap = qPoW(text, quantum_circuit, errors, num_states, num_intersection)
        error_rate = np.array(errors)/counter

        print('error rate: ' + str(error_rate) + '\n')
        pd.DataFrame(error_rate).to_csv(os.path.join(filepath, f'error_array_{n_qreg}qubit_run{counter}.csv'))
        
        with open(filename_log, 'a') as o:
            o.write(f'run {counter}' + '\n')
            o.write(f'number of qubits: {n_qreg}' + '\n')
            o.write(f'sequence of qubits used: {qubit_arr}' + '\n')
            o.write(f'backend name: {backend_name}' + '\n')
            o.write(f'number of overlap: {num_overlap}' + '\n')
            o.write('nonce: ' + str(nonce) + '\n')
            o.write('text: ' + text + '\n')
            o.write(hash_hex + '\n')
            o.write(hash_binary + '\n')
            o.write('\n')

    error_list[i] = error_rate
    pd.DataFrame(error_list[i]).to_csv(os.path.join(filepath, f'error_array_{n_qreg}qubit_final.csv'))